<div style="text-align: center; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 25px; border-radius: 10px; color: white; margin-bottom: 20px;">
  <h1 style="margin: 0; font-size: 2.2em;">SiFThon سيفثون</h1>
  <p style="margin: 10px 0 0 0; font-size: 1.1em;">منصة تنصيب سورس سيفثون</p>
</div>

## معلومات الأداة

أداة متخصصة لتنصيب سورس سيفثون على حساب التلجرام

**المميزات:**
- تشغيل فوري على سيرفر جوجل
- مدة تشغيل طويلة
- إعادة تشغيل في أي وقت
- آمن وسهل الاستخدام

## المتطلبات الأساسية

لتنصيب السورس تحتاج إلى:

**معلومات حساب التلجرام:**
- API ID من my.telegram.org
- API Hash من my.telegram.org  
- كود الجلسة من موقع توليد الجلسات

**معلومات البوت:**
- توكن البوت من BotFather

## إعداد المتغيرات

قم بإدخال البيانات المطلوبة:

In [ ]:
print("=" * 50)
print("إعداد متغيرات السورس")
print("=" * 50)

def get_user_input():
    session_code = input("ادخل كود الجلسة: ").strip()
    api_id = input("ادخل API ID: ").strip()
    api_hash = input("ادخل API Hash: ").strip()
    bot_token = input("ادخل توكن البوت: ").strip()
    
    return session_code, api_id, api_hash, bot_token

session_code, api_id, api_hash, bot_token = get_user_input()

if not all([session_code, api_id, api_hash, bot_token]):
    print("خطأ: يرجى ملء جميع الحقول")
else:
    print("تم حفظ البيانات بنجاح")

## تشغيل السورس

بعد إدخال البيانات، قم بتشغيل السورس:

In [ ]:
import os
import requests

print("=" * 50)
print("بدء تنصيب سورس سيفثون")
print("=" * 50)

def check_requirements():
    print("جاري التحقق من المتطلبات...")
    try:
        import telethon
        import pyrogram
        print("المكتبات الأساسية مثبتة")
        return True
    except ImportError:
        print("جاري تثبيت المكتبات المطلوبة...")
        return False

def install_dependencies():
    try:
        os.system("pip install telethon pyrogram tgcrypto")
        print("تم تثبيت المكتبات بنجاح")
        return True
    except Exception as e:
        print(f"خطأ في تثبيت المكتبات: {e}")
        return False

def download_sifthon():
    print("جاري تحميل السورس...")
    try:
        if not os.path.exists("SiFThon"):
            os.makedirs("SiFThon")
        
        url = "https://github.com/SOURCE-SIFTHON/SiFThon/archive/refs/heads/main.zip"
        
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open("SiFThon.zip", "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            os.system("unzip -q -o SiFThon.zip -d SiFThon_temp")
            os.system("cp -r SiFThon_temp/* SiFThon/")
            
            print("تم تحميل السورس بنجاح")
            return True
        else:
            print("فشل في تحميل السورس")
            return False
            
    except Exception as e:
        print(f"خطأ في تحميل السورس: {e}")
        return False

def create_env_file():
    print("جاري إنشاء ملف الإعدادات...")
    try:
        env_content = f"""API_ID={api_id}
API_HASH={api_hash}
SESSION={session_code}
BOT_TOKEN={bot_token}
"""
        
        with open('SiFThon/.env', 'w', encoding='utf-8') as f:
            f.write(env_content)
        
        print("تم إنشاء ملف الإعدادات")
        return True
        
    except NameError:
        print("لم يتم إدخال البيانات بعد")
        return False
    except Exception as e:
        print(f"خطأ في إنشاء ملف الإعدادات: {e}")
        return False

def run_sifthon():
    print("-" * 50)
    print("بدء تشغيل السورس...")
    print("-" * 50)
    
    try:
        os.chdir("SiFThon")
        
        print("السورس يعمل الآن...")
        print("يمكنك إيقافه بالضغط على زر Stop في الأعلى")
        
        if os.path.exists("main.py"):
            os.system("python main.py")
        elif os.path.exists("bot.py"):
            os.system("python bot.py")
        else:
            print("لم يتم العثور على ملف التشغيل الرئيسي")
            
    except Exception as e:
        print(f"خطأ في تشغيل السورس: {e}")

def main():
    try:
        if not all([session_code, api_id, api_hash, bot_token]):
            print("يرجى إدخال جميع البيانات في الخلية السابقة أولاً")
            return
    except NameError:
        print("يرجى تشغيل خلية إدخال البيانات أولاً")
        return
    
    if not check_requirements():
        if not install_dependencies():
            return
    
    if not download_sifthon():
        return
    
    if not create_env_file():
        return
    
    run_sifthon()

if __name__ == "__main__":
    main()

## استكشاف الأخطاء

إذا واجهت أي مشاكل:

In [ ]:
print("ادوات استكشاف الأخطاء")
print("=" * 40)

def troubleshoot():
    print("التحقق من الملفات:")
    if os.path.exists("SiFThon"):
        print("مجلد SiFThon موجود")
        
        files = os.listdir("SiFThon")
        if files:
            print(f"عدد الملفات: {len(files)}")
            for file in files[:5]:
                print(f"- {file}")
        else:
            print("المجلد فارغ")
    else:
        print("مجلد SiFThon غير موجود")
    
    print("التحقق من ملف الإعدادات:")
    if os.path.exists("SiFThon/.env"):
        print("ملف .env موجود")
        try:
            with open("SiFThon/.env", "r") as f:
                content = f.read()
                print("محتوى الملف:")
                print(content)
        except:
            print("لا يمكن قراءة الملف")
    else:
        print("ملف .env غير موجود")
    
    print("التحقق من المكتبات:")
    try:
        import telethon
        print("telethon مثبت")
    except:
        print("telethon غير مثبت")
    
    try:
        import pyrogram
        print("pyrogram مثبت")
    except:
        print("pyrogram غير مثبت")

troubleshoot()

<div style="text-align: center; margin-top: 30px; padding: 15px; background-color: #f8f9fa; border-radius: 5px;">
  <p style="color: #666; margin: 0;">
    SiFThon - أداة تنصيب سورس التلجرام
  </p>
</div>